In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, k_means
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.feature_selection import SelectKBest
from sklearn import metrics
from sklearn.model_selection import cross_val_score, KFold

import warnings
warnings.filterwarnings('ignore')

In [2]:
arsenal = pd.read_csv('../data/pitch-arsenal-stats.csv')
arsenal = arsenal.sort_values(by = ['last_name'], ascending = True)
spin = pd.read_csv('../data/spin-direction-pitches.csv')
spin.drop(columns = ['year'], inplace = True)
spin = spin.sort_values(by = ['last_name'], ascending = True)
pd.set_option('max_columns', None)

print(arsenal.shape)
arsenal.head()

(1227, 21)


,last_name,first_name,player_id,team_name_alt,pitch_type,pitch_name,run_value_per_100,run_value,pitches,pitch_usage,pa,ba,slg,woba,whiff_percent,k_percent,put_away,est_ba,est_slg,est_woba,hard_hit_percent
1019,Abreu,Bryan,650556,HOU,SL,Slider,1.9,5,256,37.2,63,0.246,0.456,0.338,38.0,28.6,22.5,0.253,0.423,0.332,45.0
968,Abreu,Bryan,650556,HOU,FF,4-Seamer,-0.1,0,310,45.0,66,0.273,0.418,0.379,23.9,12.1,11.4,0.274,0.428,0.367,34.0
1026,Abreu,Albert,656061,NYY,SIFT,Sinker,0.5,1,212,33.0,62,0.231,0.385,0.334,14.1,16.1,17.2,0.242,0.407,0.345,26.2
123,Adams,Austin,613534,SD,SL,Slider,-0.8,-7,838,87.1,217,0.145,0.208,0.288,35.3,33.2,28.7,0.158,0.256,0.307,23.7
600,Akin,Keegan,669211,BAL,CH,Changeup,2.8,9,344,19.5,100,0.333,0.494,0.405,29.1,16.0,17.6,0.248,0.425,0.334,38.9


In [3]:
print(spin.shape)
spin.head()

(1205, 29)


,last_name,first_name,player_id,pitch_hand,api_pitch_type,n_pitches,release_speed,spin_rate,movement_inches,alan_active_spin_pct,active_spin,hawkeye_measured,movement_inferred,api_pitch_name,active_spin_formatted,hawkeye_measured_clock_minutes,movement_inferred_clock_minutes,diff_measured_inferred,diff2,diff_measured_inferred_minutes,hawkeye_measured_clock_hh,hawkeye_measured_clock_mm,movement_inferred_clock_hh,movement_inferred_clock_mm,diff_clock_hh,diff_clock_mm,hawkeye_measured_clock_label,movement_inferred_clock_label,diff_clock_label
426,Abbott,Cory,676265,R,FF,206,92.6,2321,19.5,0.907726,0.959406,145.256367,148.948325,4-Seam Fastball,96,75,60,-3.691957,3.691957,15,1,15,1,0,0,15,1:15,1:00,+0H 15M
1055,Abreu,Bryan,650556,R,SL,256,88.9,2526,6.2,0.269459,0.324769,316.257702,314.734584,Slider,32,450,450,1.523118,1.523118,0,7,30,7,30,0,0,7:30,7:30,0H 00M
1096,Abreu,Albert,656061,R,SL,224,87.3,2209,4.5,0.206890,0.279098,196.680670,231.925560,Slider,28,690,615,-35.244890,35.244890,75,11,30,10,15,1,15,11:30,10:15,+1H 15M
419,Abreu,Bryan,650556,R,FF,308,95.7,2216,18.6,0.881274,0.925214,149.995293,153.032372,4-Seam Fastball,93,60,60,-3.037079,3.037079,0,1,0,1,0,0,0,1:00,1:00,0H 00M
925,Abreu,Albert,656061,R,SI,212,98.0,2155,20.4,1.005330,0.989575,135.456117,127.964899,Sinker,99,90,105,7.491218,7.491218,-15,1,30,1,45,0,15,1:30,1:45,-0H 15M


In [4]:
ff_arsenal = arsenal.loc[arsenal['pitch_name'] == '4-Seamer']
sl_arsenal = arsenal.loc[arsenal['pitch_name'] == 'Slider']
si_arsenal = arsenal.loc[arsenal['pitch_name'] == 'Sinker']
ch_arsenal = arsenal.loc[arsenal['pitch_name'] == 'Changeup']
cu_arsenal = arsenal.loc[arsenal['pitch_name'] == 'Curveball']
fc_arsenal = arsenal.loc[arsenal['pitch_name'] == 'Cutter']
fs_arsenal = arsenal.loc[arsenal['pitch_name'] == 'Splitter']

ff_spin = spin.loc[spin['api_pitch_type'] == 'FF']
sl_spin = spin.loc[spin['api_pitch_type'] == 'SL']
si_spin = spin.loc[spin['api_pitch_type'] == 'SI']
ch_spin = spin.loc[spin['api_pitch_type'] == 'Ch']
cu_spin = spin.loc[spin['api_pitch_type'] == 'CU']
fc_spin = spin.loc[spin['api_pitch_type'] == 'FC']
fs_spin = spin.loc[spin['api_pitch_type'] == 'FS']